In [4]:
import simpy
import random

class ClinicSimulation:
    def __init__(self, arrival_interval, treatment_mean, treatment_std, sim_duration=100):
        self.arrival_interval = arrival_interval
        self.treatment_time = {'mean': treatment_mean, 'std': treatment_std}
        self.sim_duration = sim_duration
        self.env = simpy.Environment()
        self.dentist = simpy.Resource(self.env, capacity=1)  # Changed from 'clinic' to 'dentist'
        self.served_customers = 0  # Count of served customers
    
    def customer(self, name):
        """Customer arrives, gets treated, and leaves."""
        arrival_time = self.env.now
        # print(f"{name} arrives at the clinic at {arrival_time:.2f}.")
        
        with self.dentist.request() as request:  # Changed from 'clinic' to 'dentist'
            yield request
            # print(f"{name} starts treatment at {self.env.now:.2f}.")
            yield self.env.timeout(random.gauss(self.treatment_time['mean'], self.treatment_time['std']))
            # print(f"{name} leaves the clinic at {self.env.now:.2f}.")
            self.served_customers += 1  # Increment the count of served customers

    def setup(self):
        """Create initial customers and continue to create more at random intervals."""
        i = 0
        while True:
            i += 1
            self.env.process(self.customer(f"Customer {i}"))
            yield self.env.timeout(random.expovariate(1.0 / self.arrival_interval))

    def run(self):
        """Run the clinic simulation with given parameters."""
        # print("Clinic simulation")
        random.seed(42)

        # Start the simulation process
        self.env.process(self.setup())

        # Run the simulation
        self.env.run(until=self.sim_duration)

        # print("Simulation complete.")
        print(f"Total customers served: {self.served_customers}")

# Example usage:
sim = ClinicSimulation(arrival_interval=5, treatment_mean=10, treatment_std=2, sim_duration=100)
sim.run()


Total customers served: 10


In [5]:
import simpy
import random

class Dentist:
    def __init__(self, name, can_perform_procedure_a):
        self.name = name
        self.can_perform_procedure_a = can_perform_procedure_a

class ClinicSimulation:
    def __init__(self, arrival_interval, treatment_mean, treatment_std, sim_duration=100):
        self.arrival_interval = arrival_interval
        self.treatment_time = {'mean': treatment_mean, 'std': treatment_std}
        self.sim_duration = sim_duration
        self.env = simpy.Environment()
        
        # Initialize FilterStore for dentists
        self.dentists = simpy.FilterStore(self.env)

        # Add a single dentist as an example
        self.dentists.put(Dentist("Dentist 1", can_perform_procedure_a=True))
        
        self.served_customers = 0  # Count of served customers
    
    def customer(self, name):
        """Customer arrives, gets treated, and leaves."""
        arrival_time = self.env.now
        print(f"{name} arrives at the clinic at {arrival_time:.2f}.")
        
        # Request a dentist that can perform procedure A
        dentist_request = self.dentists.get(lambda dentist: dentist.can_perform_procedure_a)
        dentist = yield dentist_request
        print(f"{name} starts treatment with {dentist.name} at {self.env.now:.2f}.")
        
        # Simulate treatment time
        yield self.env.timeout(random.gauss(self.treatment_time['mean'], self.treatment_time['std']))
        
        # Release the dentist back to the store
        self.dentists.put(dentist)
        print(f"{name} leaves the clinic at {self.env.now:.2f}.")
        self.served_customers += 1  # Increment the count of served customers

    def setup(self):
        """Create initial customers and continue to create more at random intervals."""
        i = 0
        while True:
            i += 1
            self.env.process(self.customer(f"Customer {i}"))
            yield self.env.timeout(random.expovariate(1.0 / self.arrival_interval))

    def run(self):
        """Run the clinic simulation with given parameters."""
        print("Clinic simulation")
        random.seed(42)

        # Start the simulation process
        self.env.process(self.setup())

        # Run the simulation
        self.env.run(until=self.sim_duration)

        print("Simulation complete.")
        print(f"Total customers served: {self.served_customers}")

# Example usage:
sim = ClinicSimulation(arrival_interval=5, treatment_mean=10, treatment_std=2, sim_duration=100)
sim.run()


Clinic simulation
Customer 1 arrives at the clinic at 0.00.
Customer 1 starts treatment with Dentist 1 at 0.00.
Customer 2 arrives at the clinic at 5.10.
Customer 3 arrives at the clinic at 6.36.
Customer 1 leaves the clinic at 11.58.
Customer 2 starts treatment with Dentist 1 at 11.58.
Customer 4 arrives at the clinic at 13.03.
Customer 5 arrives at the clinic at 18.68.
Customer 2 leaves the clinic at 21.84.
Customer 3 starts treatment with Dentist 1 at 21.84.
Customer 6 arrives at the clinic at 29.81.
Customer 7 arrives at the clinic at 29.96.
Customer 8 arrives at the clinic at 31.20.
Customer 3 leaves the clinic at 33.62.
Customer 4 starts treatment with Dentist 1 at 33.62.
Customer 9 arrives at the clinic at 34.72.
Customer 10 arrives at the clinic at 34.85.
Customer 11 arrives at the clinic at 35.96.
Customer 12 arrives at the clinic at 41.21.
Customer 4 leaves the clinic at 44.71.
Customer 5 starts treatment with Dentist 1 at 44.71.
Customer 13 arrives at the clinic at 45.14.
Cu

In [2]:
import simpy
import random

class Dentist:
    def __init__(self, name, can_perform_procedure_a):
        self.name = name
        self.can_perform_procedure_a = can_perform_procedure_a

class ClinicSimulation:
    def __init__(self, arrival_interval, treatment_mean, treatment_std, sim_duration=100):
        self.arrival_interval = arrival_interval
        self.treatment_time = {'mean': treatment_mean, 'std': treatment_std}
        self.sim_duration = sim_duration
        self.env = simpy.Environment()
        
        # Initialize FilterStore for dentists
        self.dentists = simpy.FilterStore(self.env)

        # Add a single dentist as an example
        self.dentists.put(Dentist("Dentist 1", can_perform_procedure_a=True))
        
        self.served_customers = 0  # Count of served customers
    
    def customer(self, name):
        """Customer arrives, gets treated, and leaves."""
        arrival_time = self.env.now
        print(f"{name} arrives at the clinic at {arrival_time:.2f}.")
        
        # Request a dentist that can perform procedure A
        dentist_request = self.dentists.get(lambda dentist: dentist.can_perform_procedure_a)
        dentist = yield dentist_request
        print(f"{name} starts treatment with {dentist.name} at {self.env.now:.2f}.")
        
        # Simulate treatment time
        yield self.env.timeout(random.gauss(self.treatment_time['mean'], self.treatment_time['std']))
        
        # Release the dentist back to the store
        self.dentists.put(dentist)
        print(f"{name} leaves the clinic at {self.env.now:.2f}.")
        self.served_customers += 1  # Increment the count of served customers

    def setup(self):
        """Create initial customers and continue to create more at random intervals."""
        i = 0
        while True:
            i += 1
            self.env.process(self.customer(f"Customer {i}"))
            yield self.env.timeout(random.expovariate(1.0 / self.arrival_interval))

    def run(self):
        """Run the clinic simulation with given parameters."""
        print("Clinic simulation")
        random.seed(42)

        # Start the simulation process
        self.env.process(self.setup())

        # Run the simulation
        self.env.run(until=self.sim_duration)

        print("Simulation complete.")
        print(f"Total customers served: {self.served_customers}")

# Example usage:
sim = ClinicSimulation(arrival_interval=5, treatment_mean=10, treatment_std=2, sim_duration=100)
sim.run()


Clinic simulation
Customer 1 arrives at the clinic at 0.00.
Customer 1 starts treatment with Dentist 1 at 0.00.
Customer 2 arrives at the clinic at 5.10.
Customer 3 arrives at the clinic at 6.36.
Customer 1 leaves the clinic at 11.58.
Customer 2 starts treatment with Dentist 1 at 11.58.
Customer 4 arrives at the clinic at 13.03.
Customer 5 arrives at the clinic at 18.68.
Customer 2 leaves the clinic at 21.84.
Customer 3 starts treatment with Dentist 1 at 21.84.
Customer 6 arrives at the clinic at 29.81.
Customer 7 arrives at the clinic at 29.96.
Customer 8 arrives at the clinic at 31.20.
Customer 3 leaves the clinic at 33.62.
Customer 4 starts treatment with Dentist 1 at 33.62.
Customer 9 arrives at the clinic at 34.72.
Customer 10 arrives at the clinic at 34.85.
Customer 11 arrives at the clinic at 35.96.
Customer 12 arrives at the clinic at 41.21.
Customer 4 leaves the clinic at 44.71.
Customer 5 starts treatment with Dentist 1 at 44.71.
Customer 13 arrives at the clinic at 45.14.
Cu

In [4]:
import simpy
import random

class Dentist:
    def __init__(self, name, can_perform_procedure_a):
        self.name = name
        self.can_perform_procedure_a = can_perform_procedure_a

class ClinicSimulation:
    def __init__(self, arrival_interval, treatment_mean, treatment_std, sim_duration=100):
        self.arrival_interval = arrival_interval
        self.treatment_time = {'mean': treatment_mean, 'std': treatment_std}
        self.sim_duration = sim_duration
        self.env = simpy.Environment()
        
        # Initialize FilterStore for dentists
        self.dentists = simpy.FilterStore(self.env)
        self.dentist_busy = 0  # Tracks how many dentists are busy at any given time
        self.dentist_busy_array = []  # Array to track busy dentists over time
        
        # Add dentists to the store
        self.dentists.put(Dentist("Dentist 1", can_perform_procedure_a=True))
        self.dentists.put(Dentist("Dentist 2", can_perform_procedure_a=False))
        
        self.served_customers = 0  # Count of served customers
    
    def customer(self, name):
        """Customer arrives and requests treatment from a capable dentist."""
        # Request a dentist that can perform procedure A
        dentist_request = self.dentists.get(lambda dentist: dentist.can_perform_procedure_a)
        dentist = yield dentist_request

        # Increment busy count and start treatment
        self.dentist_busy += 1
        self.monitor_dentist()  # Monitor the busy state
        yield self.env.timeout(random.gauss(self.treatment_time['mean'], self.treatment_time['std']))
        
        # Release the dentist and decrement busy count
        self.dentists.put(dentist)
        self.dentist_busy -= 1
        self.served_customers += 1

    def monitor_dentist(self):
        """Function to monitor and record the number of busy dentists."""
        while True:
            self.dentist_busy_array.append(self.dentist_busy)
            yield self.env.timeout(1)
    
    def setup(self):
        """Create initial customers and continue to create more at random intervals."""
        i = 0
        while True:
            i += 1
            self.env.process(self.customer(f"Customer {i}"))
            yield self.env.timeout(random.expovariate(1.0 / self.arrival_interval))

    def run(self):
        """Run the clinic simulation with given parameters."""
        random.seed(42)
        self.env.process(self.setup())
        self.env.process(self.monitor_dentist())
        self.env.run(until=self.sim_duration)
        

# Example usage:
sim = ClinicSimulation(arrival_interval=5, treatment_mean=10, treatment_std=2, sim_duration=100)
sim.run()

# Output the dentist_busy_array to see how many dentists were busy over time
print(sim.dentist_busy_array)


[0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [5]:
competence_matrix = {
    "Dentist 1": {"Procedure A", "Procedure B"},
    "Dentist 2": {"Procedure B", "Procedure C"},
    "Dentist 3": {"Procedure A", "Procedure C"}
}

# Populate FilterStore with dentists based on their competencies
for name, competencies in competence_matrix.items():
    print(name, competencies)

Dentist 1 {'Procedure B', 'Procedure A'}
Dentist 2 {'Procedure B', 'Procedure C'}
Dentist 3 {'Procedure C', 'Procedure A'}


In [11]:
import simpy
import random

class Dentist:
    def __init__(self, name, competencies):
        self.name = name
        self.competencies = competencies  # Set or list of procedures the dentist can perform

class ClinicSimulation:
    def __init__(self, arrival_interval, treatment_mean, treatment_std, sim_duration=100):
        self.arrival_interval = arrival_interval
        self.treatment_time = {'mean': treatment_mean, 'std': treatment_std}
        self.sim_duration = sim_duration
        self.env = simpy.Environment()
        
        # Initialize FilterStore for dentists
        self.dentists = simpy.FilterStore(self.env)
        self.dentist_busy = 0  # Tracks how many dentists are busy at any given time
        self.dentist_busy_array = []  # Array to track busy dentists over time
        
        # Example matrix of competencies
        competence_matrix = {
            "Dentist 1": {"Procedure A", "Procedure B"},
            "Dentist 2": {"Procedure B", "Procedure C"},
            "Dentist 3": {"Procedure A", "Procedure C"}
        }

        # Populate FilterStore with dentists based on their competencies
        for name, competencies in competence_matrix.items():
            self.dentists.put(Dentist(name, competencies))
        
        self.served_customers = 0  # Count of served customers
    
    def customer(self, name, procedure):
        """Customer arrives and requests treatment for a specific procedure."""
        # Request a dentist who can perform the required procedure
        dentist_request = self.dentists.get(lambda dentist: procedure in dentist.competencies)
        dentist = yield dentist_request

        # Increment busy count and start treatment
        self.dentist_busy += 1
        yield self.env.timeout(random.gauss(self.treatment_time['mean'], self.treatment_time['std']))
        
        # Release the dentist and decrement busy count
        self.dentists.put(dentist)
        self.dentist_busy -= 1
        self.served_customers += 1

    def monitor_dentist(self):
        """Function to monitor and record the number of busy dentists."""
        while True:
            self.dentist_busy_array.append(self.dentist_busy)
            yield self.env.timeout(1)

    def setup(self):
        """Create initial customers and continue to create more at random intervals."""
        i = 0
        procedures = ["Procedure A", "Procedure B", "Procedure C"]
        while True:
            i += 1
            procedure = random.choice(procedures)  # Randomly assign a procedure for demonstration
            self.env.process(self.customer(f"Customer {i}", procedure))
            yield self.env.timeout(random.expovariate(1.0 / self.arrival_interval))

    def run(self):
        """Run the clinic simulation with given parameters."""
        random.seed(42)
        self.env.process(self.setup())
        self.env.process(self.monitor_dentist())
        self.env.run(until=self.sim_duration)

# Example usage:
sim = ClinicSimulation(arrival_interval=5, treatment_mean=10, treatment_std=2, sim_duration=100)
sim.run()

# Output the dentist_busy_array to see how many dentists were busy over time
print(sim.dentist_busy_array)


[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [16]:
import pandas as pd

# Example matrix of competencies
competence_matrix = {
    "Dentist 1": {"Procedure A", "Procedure B"},
    "Dentist 2": {"Procedure B", "Procedure C"},
    "Dentist 3": {"Procedure A", "Procedure C"}
}

In [24]:
# Convert to a DataFrame using from_dict and stack/unstack
df = pd.DataFrame.from_dict(competence_matrix, orient='index').stack().reset_index(level=1, drop=True).reset_index()
df.columns = ['Dentist', 'Procedure']

# Convert it to a wide format with binary indicators
df['Value'] = 1
df = df.pivot(index='Dentist', columns='Procedure', values='Value').fillna(0).astype(int)
df

Procedure,Procedure A,Procedure B,Procedure C
Dentist,,,
Dentist 1,1,1,0
Dentist 2,0,1,1
Dentist 3,1,0,1


In [26]:
import pandas as pd

# Manual creation of DataFrame
dentists = list(competence_matrix.keys())
procedures = list(set(procedure for procedures in competence_matrix.values() for procedure in procedures))

data = []
for dentist in dentists:
    row = {procedure: 1 if procedure in competence_matrix[dentist] else 0 for procedure in procedures}
    row['Dentist'] = dentist
    data.append(row)

df = pd.DataFrame(data).set_index('Dentist')
df


,Procedure B,Procedure A,Procedure C
Dentist,,,
Dentist 1,1,1,0
Dentist 2,1,0,1
Dentist 3,0,1,1
